# Dědičnost

In [ ]:
import numpy as np

class Shape:
    def __init__(self, name):
        self.name = name
        
    def area(self):
        pass
    
    def circumference(self):
        pass
    
    def __str__(self):
        return """{}, {}
        Obvod: {}
        Obsah: {}
        """.format(self.name, self.__class__.__name__, self.circumference(), self.area())

class Rectangle(Shape):
    # overriding
    def __init__(self, name, a, b):
        self.a = a
        self.b = b
        super().__init__(name)
    
    def area(self):
        return self.a * self.b
    
    def circumference(self):
        return 2.0 * (self.a + self.b)
    
class Square(Rectangle):
    def __init__(self, name, a):
        super().__init__(name, a, a)
        
class Circle(Shape):
    pass


class Polygon(Shape):
    def __init__(self, name, a, n):
        self.a = a
        self.n = n
        super().__init__(name)
        
    def area(self):
        # S = a^2 * n / (4 tan(pi/n))
        return self.a**2 * self.n / (4* np.tan(np.pi / self.n))

    def circumference(self):
        return self.a * self.n

class Hexagon(Polygon):
    def __init__(self, name, a):
        super().__init__(name, a, 6)
        
    def __repr__(self):
        return "{}('{}', {:.5f})".format(self.__class__.__name__, self.name, self.a)

In [ ]:
r = Rectangle("obd1", 1, 2)
s = Square("ctv1", 2)
p = Polygon("pol1", 2, 4)
h = Hexagon("hex1", 1)
print(p) # print automaticky zavola str(p)
print(s)

rep = repr(h)

h2 = eval(rep)
print(h2)

## Abstraktni tridy

In [ ]:
from abc import ABC, abstractmethod # Abstract Base Class
class Shape(ABC):
    def __init__(self, name):
        self.name = name
    
    @abstractmethod
    def area(self):
        pass
    
    @abstractmethod
    def circumference(self):
        pass
    
    def __str__(self):
        return """{}, {}
        Obvod: {}
        Obsah: {}
        """.format(self.name, self.__class__.__name__, self.circumference(), self.area())

class Rectangle(Shape):
    # overriding
    def __init__(self, name, a, b):
        self.a = a
        self.b = b
        super().__init__(name)
    
    def area(self):
        return self.a * self.b

    def circumference(self):
        return 2.0 * (self.a + self.b)
    
r = Rectangle("r", 2, 3)
print(r)

In [ ]:
class A:
    """
    This is class A
    """
    @staticmethod
    def f(x: int):
        """This is a static method of class A
        Args:
            x = number of cats you have at home
        """
        pass

print(A.__doc__)
a = A()
A.f.__doc__

# Vlastni vyjimky

In [ ]:
try:
    1 /0
except ArithmeticError as e:
    # Exception -> ... -> ArithmeticError -> ZeroDivisionError
    print(e, type(e))

In [ ]:
from time import sleep
filename = "config.ini"

def create_default_file():
    sleep(2)
    
try:
    file = open(filename, "r")
except Exception as e:
    print(e)
    print("Creating default file...", end ="")
    create_default_file()
    print("done")
    print("resuming")
    
# zbytek programu

In [ ]:
class ConfigGone(Exception):
    def __init__(self, filename, message = "Confing file not found"):
        self.filename = filename
        self.message = message
        super().__init__(message)
        
    def __str__(self):
        return f"{self.filename}: {self.message}"

In [ ]:
from time import sleep
import os.path
import numpy as np

filename = "config.ini"

def create_default_file():
    sleep(2)
    
try:
    if np.random.randint(10) > 1:
        raise Exception("a jeje")
        
    if not os.path.isfile(filename):
        raise ConfigGone(filename)
        
except ConfigGone as e:
    print(e, type(e))
    print("Creating default file...", end ="")
    create_default_file()
    print("done")
    print("resuming")
except Exception as e:
    print(e)
    
# zbytek programu

## Navrhove vzory ( design patterns)
### Observer pattern (publisher-subscriber pattern)

In [ ]:
class Subscriber:
    def __init__(self, name):
        self.name = name
    
    def update(self, message):
        print("{} got message: {}".format(self.name, message))
    
class Publisher:
    def __init__(self):
        self.subscribers = []
    
    def register(self, who: Subscriber):
        self.subscribers.append(who)
    
    def unregister(self, who: Subscriber):
        self.subscribers.remove(who)
    
    def dispatch(self, message):
        # updates all subscribers
        for subscriber in self.subscribers:
            subscriber.update(message)
            
sub1 = Subscriber("Vaclav")
sub2 = Subscriber("Karel")

pub = Publisher()
pub.register(sub1)

pub.dispatch("Prvni zprava")

pub.register(sub2)

pub.dispatch("Druha zprava")

pub.unregister(sub1)
pub.dispatch("Treti zprava")

In [ ]:
import numpy as np
import time

class FireDetector(Publisher):
    def __init__(self, threshold = 100):
        super().__init__()
        self.threshold = threshold
        self.extinguishing = False
        
    def read_temperature(self):
        return np.random.randint(200)
    
    def run(self):
        for i in range(10):
            temp = self.read_temperature()
            print("Temperature is:", temp)
            if temp > self.threshold and not self.extinguishing:
                self.dispatch(True)
                self.extinguishing = True
            if temp <= self.threshold and self.extinguishing:
                self.dispatch(False)
                self.extinguishing = False
            time.sleep(2)
            
class Extinguisher(Subscriber):
    def __init__(self, name):
        super().__init__(name)
    
    def update(self, switch):
        if switch:
            print(f"\tExtinguisher {self.name} is on")
        else:
            print(f"\tExtinguisher {self.name} is off")
            
class ThePentagon(Polygon, Subscriber):
    def __init__(self):
        Subscriber.__init__(self, "The Pentagon")
        Polygon.__init__(self, "The Pentagon", 300, 5)

    def update(self, fire):
        if fire:
            print("\tThe Pentagon knows there is a fire")
        else:
            print("\tThe Pentagon knows there is no fire")
        
        
pentagon = ThePentagon()

print(pentagon)
det1 = FireDetector()

ext1 = Extinguisher("ext1")
ext2 = Extinguisher("ext2")

det1.register(ext1)
det1.register(ext2)
det1.register(pentagon)

det1.run()

## dotaz - pretizeni metody
- v pythonu nemame (mame neco jineho)

- C++ umoznuje pretezovani funkci (metod), tj. muzu mit dve funkce se stejnym nazvem a jinou signaturou
```C++
int add(int x, int y) {
    return x + y;
}

double add(double x, double y) {
    return x + y;
}
```

```C
int add_i(int x, int y) {
    return x + y;
}

double add_d(double x, double y) {
    return x + y;
}
```